In [229]:
import numpy as np
import pandas as pd
import random

In [230]:
cities = ['A', 'B', 'C', 'D', 'E']


In [231]:
distances = [[ 0.00, 28.02, 17.12, 27.46, 46.07],
            [28.02,  0.00, 34.00, 25.55, 25.55],
            [17.12, 34.00,  0.00, 18.03, 57.38],
            [27.46, 25.55, 18.03,  0.00, 51.11],
            [46.07, 25.55, 57.38, 51.11,  0.00]]

# Initialize Population

In [232]:
def initializePopulation():
    population = []
    for i in range(10):
        randomSolution = cities.copy()
        random.shuffle(randomSolution)
        population.append(randomSolution)
    return np.array(population)

# Evaluation and Selection

In [233]:
def fitnessFunction(solution):
    totalDistance = 0
    for i in range(len(solution)-1):
        totalDistance += distances[cities.index(solution[i])][cities.index(solution[i+1])]

    return totalDistance

In [234]:
#Roulette Wheel Selection (others include Rank method and Tournament Size)
def evaluatePopulation(population):

    fitnessValues = []
    for solution in population:
        fitnessValues.append(fitnessFunction(solution))

    centeredValues = [np.max(list(fitnessValues))-i for i in list(fitnessValues)]
    weightedValues = [i/sum(centeredValues) for i in centeredValues]


    return fitnessValues, weightedValues


def RouletteWheel(population, weightedFitness):
    
    while (True):
        randomIndex = random.randint(0, len(population)-1)
        randomChance = weightedFitness[randomIndex]
        
        if random.random() <= randomChance:
            pickedSolution = population[randomIndex]
            break
    
    return pickedSolution

# Combination and Mutation

In [235]:
def crossover(solution1, solution2):
    n = len(solution1)
    num_els = np.ceil(n*(random.randint(10,90)/100))

    child = [np.nan for i in range(n)]
    str_pnt = random.randint(0, n-2)
    end_pnt = n if int(str_pnt+num_els) > n else int(str_pnt+num_els)
    blockA = list(solution1[str_pnt:end_pnt])
    child[str_pnt:end_pnt] = blockA
    
    for i in range(n):
        if list(blockA).count(solution2[i]) == 0:
            for j in range(n):
                if pd.isna(child[j]):
                    child[j] = solution2[i]
                    break
    return child

In [236]:
def swap(solution, position1, position2):
    
    result = solution.copy()
    result[position1] = solution[position2]
    result[position2] = solution[position1] 
    return result

def swapMutation(solution):
    
    n = len(solution)
    pos_1 = random.randint(0,n-1)
    pos_2 = random.randint(0,n-1)
    
    return swap(solution, pos_1, pos_2)

# Main

In [237]:
generations = 200

In [238]:
#INITIALIZE population
population = initializePopulation()

#EVALUATE initial poplulation
populationFitness, weightedFitness = evaluatePopulation(population)
globalBestFitness = np.min(populationFitness)
bestIndex = populationFitness.index(globalBestFitness)
globalBestSolution = population[bestIndex]

#Begin Evolution Search
for generation in range(generations):

    newPopulation = []
    for i in range(len(population)):

        # SELECT Parents
        parent1 = RouletteWheel(population, weightedFitness)
        parent2 = RouletteWheel(population, weightedFitness)
        offspring = parent1

        #RECOMBINE Parents
        if random.random() <= .87:
            offspring = crossover(parent1, parent2)

        #MUTATE Offspring
        if random.random() <= .7:
            offspring = swapMutation(offspring)

        newPopulation.append(offspring)

    #Evaluate newPopulation
    populationFitness, weiwhtedFitness = evaluatePopulation(newPopulation)

    bestFitness = np.min(populationFitness)
    bestIndex = populationFitness.index(bestFitness)
    bestSolution = newPopulation[bestIndex]

    if bestFitness <= globalBestFitness:
        globalBestFitness = bestFitness
        globalBestSolution = bestSolution

    #SELECT Survival
    population = newPopulation

In [239]:
print(globalBestSolution)

['A', 'C', 'D', 'B', 'E']
